In [1]:
import cv2
import itertools,os
import json
import numpy as np
from Model import get_Model
from parameter import letters
from keras import backend as K
from tqdm import tqdm
K.set_learning_phase(0)

Using TensorFlow backend.


In [2]:
def decode_label(out):
    # out : (1, 32, 42)
    out_best = list(np.argmax(out[0, 2:], axis=1))  # get max index -> len = 32
    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value
    outstr = ''
    for i in out_best:
        if i < len(letters):
            outstr += letters[i]
    return outstr

In [3]:
weightpath='LSTM+BN5--10--29.857.hdf5'
test_dir="test/testimage/"
# Get CRNN model
model = get_Model(training=False)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 256, 32, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 32, 64)  640         the_input[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 32, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 32, 64)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max1 (MaxP

In [4]:
try:
    model.load_weights(weightpath)
    print("...Previous weight data...")
except:
    raise Exception("No weight file!")

...Previous weight data...


In [6]:
dicts={}
test_imgs = os.listdir(test_dir)
for test_img in tqdm(test_imgs):
    img = cv2.imread(test_dir + test_img, cv2.IMREAD_GRAYSCALE)
    img_pred = img.astype(np.float32)
    img_pred = cv2.resize(img_pred, (256,32))
    img_pred = (img_pred / 255.0) * 2.0 - 1.0
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)
    net_out_value = model.predict(img_pred)
    pred_texts = decode_label(net_out_value)
    dicts[test_img[:-4]]=pred_texts

100%|██████████| 977/977 [01:09<00:00, 14.02it/s]


In [23]:
print(len(dicts))

977


# 将结果写入json,由于直接dump一直出乱码，因此用了一个笨办法重写json

In [24]:
json_path = 'test/ocrtest/submission_sample_for_Recognition.json'
new_json = 'test/ocrtest/submission.json'
totalnum=int(2931/3)
with open(json_path, 'r') as f:
    json_data = json.load(f)
    a=json_data
    for i in range(totalnum):
        for key in a[i]:
            if key in dicts:
                a[i][key]=dicts[key]
            else:
                print(key)
f.close()

def getKeyandValue(a):
    for k in a:
        return k,a[k]
    
f=open(new_json,'w')
f.write('[\n')
for i in range(totalnum-1):
    f.write('    {\n')
    key,value = getKeyandValue(a[i])
    f.write('        "'+key+'":"'+value+'"\n')
    f.write('    },\n')
    
f.write('    {\n')
key,value = getKeyandValue(a[totalnum-1])
f.write('        "'+key+'":"'+value+'"\n')
f.write('    }\n]')
f.close()